<a href="https://colab.research.google.com/github/edcalderin/DeepLearning_SaturdaysAI/blob/master/Tareas/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea: Recurrent Neural Networks
### Grupo: XXXXX
Integrantes:
<br>
* Integrante 1 
* Integrante 2
* Integrante 3
* Integrante 4
<br>

Indicaciones:
<rb>
* Debe realizar la siguiente tarea hasta el miercoles 16 de junio, 23:59 UTC - 4
* Debe hacer una copia de este notebook para poder editar el código.
* Debe poner el código faltante en las celdas que correspondan.
* Una vez finalizado el trabajo debe subir el link de su notebook (con permisos de lector) en la sección de "Tareas" del Módulo 3: Recurrent Neural Networks en Eduflow.

En la parte práctica de la clase, vimos cómo entrenar una red neuronal recurrente (RNN) para la tarea del análisis del sentimiento. Sin embargo, los resultados en el test set no fueron nada buenos. En este ejercicio, haremos unos cuantos cambios para crear un modelo que nos dé una precisión de más del 80%:



*   Cambiar el RNN por un **LSTM bidireccional**
*   Utilizar **pre-trained word embeddings** españoles
*   Regularización
*   Un optimizer distinto
*   Procesaremos solo los elementos que no son *padding* (**packed padded sequences**)

Para utilizar los embeddings, hay que bajarse el fichero de https://www.kaggle.com/rtatman/pretrained-word-vectors-for-spanish tal y como vimos en la parte práctica. 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Bajarse el tokenizer español de SpaCy

!python -m spacy download es_core_news_sm

Recordad que después de bajarse el tokenizer, para que funcione correctamente hay que reanudar el runtime.

*Runtime -> Restart runtime*

# Dataset

Vamos a inizializar el dataset en torchtext tal y como hicimos en las prácticas. Sin embargo, vamos a añador un parámetro extra en *data.Field* de TEXT llamado *include_lengths=True*. Para poder procesar los elementos que no son padding, necesitamos saber la longitud de cada texto en el dataset.  

In [ ]:
# Inicializar torchtext dataset

import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = ## SU CÓDIGO AQUÍ ##

SENTIMENT = ## SU CÓDIGO AQUÍ ##

In [ ]:
# Indicar a torchtext qué campos corresponden a los distintos elementos del json

fields = ## SU CÓDIGO AQUÍ ##

Ahora vamos a leer el corpus. Vamos a utilizar los tres ficheros (**train.json**, **valid.json** y **test.json**) que vimos en la parte práctica. 

In [ ]:
# Leer corpus

PATH = 'drive/MyDrive/Saturdays.AI/data_sentimiento'

train_data, valid_data, test_data = ## SU CÓDIGO AQUÍ ##

Como vamos a utilizar *pre-trained embeddings*, tenemos que añadirlos cuando construimos el vocabulario. Para inicializar el vocabulario en los pre-trained embeddings a 0, añadimos el parámetro *unk_init*.

In [ ]:
# Leer embeddings, este proceso puede tardar unos segundos

import torchtext.vocab as vocab

FILE_NAME = 'SBW-vectors-300-min5.txt'
PATH = ## SU PATH AQUÍ ##

spanish_embeddings = vocab.Vectors(FILE_NAME, cache=PATH)

In [ ]:
# Constuir vocabulario 

MAX_VOCAB_SIZE = 4000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, 
                 vectors = spanish_embeddings, 
                 unk_init = torch.Tensor.normal_)

SENTIMENT.build_vocab(train_data)

In [ ]:
# Preparar train, valid y test iterators para entrenar el modelo

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = ## SU CÓDIGO AQUÍ ##

In [ ]:
# Para ver la dimensión de los pre-trained embeddings

pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([4002, 300])


# Modelo

Construimos el modelo que entrenaremos y evaluaremos. En esta tarea utilizamos un **LSTM bidireccional**. 

La *embedding* layer tendrá un parámetro extra, ***padding_idx=pad_idx***, que indica el índice del token *pad* para que el model no lo procese.

La *rnn* layer será ahora de tipo **nn.LSTM** con los parámetros siguientes:


*   **embedding_dim**: dimensión de los pre-trained embeddings
*   **hidden_dim**: dimensión de la hidden layer
*   **num_layer**: número de layers
*   **bidirectional**: queremos un LSTM bidireccional 
*   **dropout**: el dropout para regularizar la red neuronal

Luego añadimos una **layer linear** y *dropout*. La dimensión de la hidden layer que pasamos por la linear layer es el **doble** porque concatenamos las dos hidden layers con distintas direcciones. 

También vamos a definir el paso forward. Atención que utilizamos *dropout* y *packed_padded_sequence*. Al final concatenamos las dos hidden layers del Bidirectional LSTM. 



In [ ]:
# Construir modelo

import torch.nn as nn

class BiLSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
    super().__init__()

    self.embedding = ## SU CÓDIGO AQUÍ ##

    self.rnn = ## SU CÓDIGO AQUÍ ##
    
    self.fc = ## SU CÓDIGO AQUÍ ##

    self.dropout = ## SU CÓDIGO AQUÍ ##

    # Forward pass
  
  def forward(self, text, text_lengths):

    embedded = self.dropout(self.embedding(text))

    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
    
    packed_output, (hidden, cell) = self.rnn(packed_embedded)

    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

    hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))

    return self.fc(hidden)

In [ ]:
# Definimos parámetros, algunos (como hidden_dim o n_layers) los podéis cambiar y experimentar

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 450
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Inicializamos el modelo con todos los parámetros

## SU CÓDIGO AQUÍ ##

In [ ]:
# Para ver el número de parámetros que entrenaremos en la red neuronal

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model):,} parámetros')

Finalmente, copiamos los pre-trained word embeddings y los metemos en la *embedding layer*. Luego reemplazamos los valores iniciales de la *embedding layer* con los pre-trained embeddings. 

In [ ]:
# Copiamos pre-trained embeddings

pretrained_embeddings = TEXT.vocab.vectors

# Inicializamos embedding layer

model.embedding.weight.data.copy_(TEXT.vocab.vectors)

# Entrenar 

Vamos a entrenar el modelo. Empezamos definiendo el *optimizer*. Esta vez vamos a utilizar **Adam**. Utilizamos la misma loss function que vimos en la parte práctica.

In [ ]:
# Optimizer

import torch.optim as optim

optimizer = ## SU CÓDIGO AQUÍ ##

# loss function

criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Modelo y loss function en GPU

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
# función para calcular accuracy 

def binary_accuracy(preds, y):
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float()
  acc = correct.sum() / len(correct)
  return acc

Definimos la función para entrenar el modelo. Como que hemos incluido el parámetro *include_lengths=True*, en este caso *batch.t* es una tupla con el primer elemento una tensor de números y el segundo elemento la longitud de cada texto. Con lo cual, antes de pasar *batch.t*, tendremos que separar estos dos elementos:

**text, text_lengths = batch.t**

Y pasar las dos variables (*text* y *text_lengths*) al modelo. 

In [ ]:
def train(model, iterator, optimizer, criterion):

  ## SU CÓDIGO AQUÍ ##
  
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

Para definir la función que evalúa el modelo, recordar que es muy similar a *train*, con alguna diferencia (ver ejercicio práctico). 

In [ ]:
def evaluate(model, iterator, criterion):
    
    ## SU CÓDIGO AQUÍ ##
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# Función para saber el tiempo que se tarda para entrenar cada epoch

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Vamos a entrenar tal y como lo hicimos en el ejercicio práctico. 

In [ ]:
# Entrenamos

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    ## SU CÓDIGO AQUÍ ##
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# Test

Vamos a ver la precisión del modelo en el test data. Deberías obtener una accuracy alrededor del 80%!

In [ ]:
# Resultados en el test set

MODEL_NAME = ## SU NOMBRE AQUÍ ##

model.load_state_dict(torch.load(MODEL_NAME))

test_loss, test_acc = ## SU CÓDIGO AQUÍ ##

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Si quieres ver cómo funciona el modelo con tus propios comentarios positivos o negativos, podemos crear una función para hacer predicciones.  

In [ ]:
# Cargar el tokenizer de SpaCy

import spacy
nlp = spacy.load('es_core_news_sm')

# Función para predecir sentimiento 

def predict_sentiment(model, sentence):
    # modelo en modo evaluación
    model.eval()
    # tokenizar texto
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    # transformar palabras en sus índices del vocabulario
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    # convertir lista de índices en tensor
    tensor = torch.LongTensor(indexed).to(device)
    # añadir una dimensión para batch
    tensor = tensor.unsqueeze(1)
    # convertir length en un tensor
    length_tensor = torch.LongTensor(length)
    # predecir, utilizando sigmoid para obtener un número entre 0 y 1
    prediction = torch.sigmoid(model(tensor, length_tensor))
    if prediction.item() >= 0.5:
      return "negativo"
    else:
      return "positivo"

In [ ]:
# Podéis probar con el texto que queráis. 

TEXTO = "Servicio excelente!"

predict_sentiment(model, TEXTO)